In [21]:
import http
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time

from selenium.common.exceptions import TimeoutException, NoSuchElementException, StaleElementReferenceException

In [22]:
def micrhaz_type(driver):
    """
    function for choosing the all michrazim types
    """
    micrhaz_type_button = driver.find_element(By.XPATH,
                                              '//*[@class="p-multiselect-trigger-icon ng-tns-c30-0 pi pi-chevron-down"]')
    micrhaz_type_button.click()
    time.sleep(0.2)
    driver.find_elements(By.XPATH, '//*[@class="p-checkbox-box ng-tns-c30-0"]')[0].click()

In [23]:
def region_rmi(driver):
    """
    function for choosing the all eras types
    """
    micrhaz_merhav_type = driver.find_element(By.XPATH,
                                              '//*[@class="p-multiselect-trigger-icon ng-tns-c30-3 pi pi-chevron-down"]')  #מרחב ברמ"י
    micrhaz_merhav_type.click()
    time.sleep(0.2)
    driver.find_elements(By.XPATH, '//*[@class="ng-star-inserted"]')[6].click()
    # driver.find_elements(By.XPATH, '//*[@class="p-checkbox-box ng-tns-c30-3"]')[4].click()

In [24]:
def micrhaz_purpose(driver):
    """
   יעוד מכרז -  בנייה נמוכה/צמודת קרקע
   """
    micrhaz_yehud_type = driver.find_element(By.XPATH,
                                             '//*[@class="p-multiselect-trigger-icon ng-tns-c30-2 pi pi-chevron-down"]')
    micrhaz_yehud_type.click()
    time.sleep(0.2)
    driver.find_elements(By.XPATH, '//*[@class="ng-star-inserted"]')[1].click()  #בנייה נמוכה/צמודת קרקע

In [25]:
def micrhaz_date(driver):
    """
    funcrion for choosing the start date serach
    """
    micrhaz_date_result = driver.find_element(By.XPATH,
                                              '//*[@class="ng-tns-c34-8 p-inputtext p-component ng-star-inserted"]')
    micrhaz_date_result.send_keys(Keys.CONTROL + 'a' + Keys.NULL, '01/01/2010')
    time.sleep(2)
    
    micrhaz_date_result = driver.find_element(By.XPATH,
                                              '//*[@class="ng-tns-c34-7 p-inputtext p-component ng-star-inserted"]')
    micrhaz_date_result.send_keys(Keys.CONTROL + 'a' + Keys.NULL, '01/01/2005' + Keys.ENTER)
    time.sleep(8)

In [26]:
### DATA FRAME EXPLANATION ###
"""
df = {
    "Tender Number": "מספר מכרז",
    "Region": "מרחב ברמ\"י",
    "City": "יישוב",
    "Purpose": "ייעוד",
    "Status": "סטטוס",
    "Publication date": "פרסום",
    "Opening date": "פתיחה",
    "Committee date": "ועדת מכרזים / תאריך ועדת מכרזים",
    "Submission deadline": "מועד אחרון להגשת הצעות",
    "Target audience": "עדיפות לקהל יעד",

    "Compound Number": "מספר מתחם",
    "Units": "מספר יחידות דיור",
    "Winner Name": "שם זוכה",
    "Final Price": "מחיר סופי בשקלים",
    "Development Cost": "הוצאות פיתוח בשקלים",
    "Valuation Price": "מחיר שומה",
    "Minimum Price": "מחיר מינימום",
    "Area m2": "שטח במטר מרובע",

    "Plan": "תוכנית",
    "Parcel": "מגרש",
    "Block": "גוש",
    "Section": "חלקה",

}
"""


'\ndf = {\n    "Tender Number": "מספר מכרז",\n    "Region": "מרחב ברמ"י",\n    "City": "יישוב",\n    "Purpose": "ייעוד",\n    "Status": "סטטוס",\n    "Publication date": "פרסום",\n    "Opening date": "פתיחה",\n    "Committee date": "ועדת מכרזים / תאריך ועדת מכרזים",\n    "Submission deadline": "מועד אחרון להגשת הצעות",\n    "Target audience": "עדיפות לקהל יעד",\n\n    "Compound Number": "מספר מתחם",\n    "Units": "מספר יחידות דיור",\n    "Winner Name": "שם זוכה",\n    "Final Price": "מחיר סופי בשקלים",\n    "Development Cost": "הוצאות פיתוח בשקלים",\n    "Valuation Price": "מחיר שומה",\n    "Minimum Price": "מחיר מינימום",\n    "Area m2": "שטח במטר מרובע",\n\n    "Plan": "תוכנית",\n    "Parcel": "מגרש",\n    "Block": "גוש",\n    "Section": "חלקה",\n\n}\n'

In [27]:
def scroll_to_load_all_results(driver, max_scrolls=20, scroll_pause_time=1,type=1):
    previous_count = 0
    no_change_counter = 0

    for i in range(max_scrolls):
        # Scroll to the bottom of the page
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait for new results to load
        time.sleep(scroll_pause_time)

        # Count current number of loaded tender results
        if type==0:
            
            results = driver.find_elements(By.XPATH,
                '//div[contains(@class, "list-rows list-group-item align-left ng-star-inserted")]')        
        else :
            results = driver.find_elements(By.CSS_SELECTOR,'div.tik-record-wrapper')

        current_count = len(results)

        print(f"Scroll {i + 1}: found {current_count} tenders")

        # Check if no new results are loaded
        if current_count == previous_count:
            no_change_counter += 1
            if no_change_counter >= 3:
                print("No more results found. Scrolling finished.")
                break
        else:
            no_change_counter = 0

        previous_count = current_count


In [28]:
def extract_tender_data(driver):

    fields_mapping = {
        "מרחב ברמ\"י": "Region",
        "יישוב": "City",
        "ייעוד": "Purpose",
        "סטטוס": "Status",
        "פרסום": "Publication date",
        "פתיחה": "Opening date",
        "מועד אחרון להגשת הצעות": "Submission deadline",
        "ועדת מכרזים": "Committee date",
        "תאריך ועדת מכרזים": "Committee date",
        "עדיפות לקהל יעד": "Target audience"
    }

    tender_data = {}

    # Extract tender number
    try:
        tender_number_element = driver.find_element(By.CLASS_NAME, "mis-michraz")
        tender_number = tender_number_element.text.strip()
        tender_data["Tender Number"] = tender_number
        print(f"📄 Tender Number: {tender_number}")
    except:
        print("⚠️ Could not extract Tender Number")
        tender_data["Tender Number"] = None

    # Extract general tender fields
    all_d_flex = driver.find_elements(By.XPATH, '//div[contains(@class, "d-flex")]')
    for div in all_d_flex:
        try:
            title_span = div.find_element(By.XPATH, './/span[@class="title"]')
            field_name = title_span.text.replace(':', '').strip()
            value_span = div.find_element(By.XPATH, './/span[@tabindex="0"]')
            field_value = value_span.text.strip()

            for hebrew_key, english_key in fields_mapping.items():
                if hebrew_key in field_name:
                    tender_data[english_key] = field_value
                    print(f"✅ Found field: '{english_key}' = '{field_value}'")
                    break
            else:
                tender_data[field_name] = field_value
                print(f"📎 Unmapped field: '{field_name}' = '{field_value}'")

        except Exception as e:
            continue

    # Extract info for each "mitcham"
    
    scroll_to_load_all_results(driver)
    mitcham_elements = driver.find_elements(By.CSS_SELECTOR, 'div.tik-record-wrapper')
    all_mitcham_data = []
    counter=0
    for mitcham in mitcham_elements:
        time.sleep(2)
        try:
            # Expand hidden section
            try:
                expand_btn = mitcham.find_element(By.CLASS_NAME, 'showAllIcon')
                driver.execute_script("arguments[0].click();", expand_btn)
                time.sleep(1)
            except Exception as e:
                print("⚠️ Could not expand mitcham section:", e)

            mitcham_data = tender_data.copy()

            def get_xpath_text(xpath):
                try:
                    value = mitcham.find_element(By.XPATH, xpath).text.strip()
                    print(f"✅ Loaded: {xpath} = {value}")
                    return value
                except:
                    print(f"⛔ Missing: {xpath}")
                    return "N/A"

            mitcham_data["Compound Number"] = get_xpath_text('.//label[contains(text(),"מספר מתחם")]/following-sibling::label[1]')
            mitcham_data["Units"] = mitcham.find_element(By.XPATH, ".//label[contains(text(),'יח\"ד')]/following-sibling::label[1]").text.strip()
            mitcham_data["Winner Name"] = get_xpath_text('.//label[contains(text(),"שם זוכה")]/following-sibling::label[1]')
            mitcham_data["Final Price"] = get_xpath_text('.//label[contains(text(),"מחיר סופי")]/following-sibling::label[1]')
            mitcham_data["Development Cost"] = get_xpath_text('.//label[contains(text(),"הוצאות פיתוח")]/following-sibling::label[1]')
            mitcham_data["Valuation Price"] = get_xpath_text('.//label[contains(text(),"שומה")]/following-sibling::label[1]')
            mitcham_data["Minimum Price"] = get_xpath_text('.//label[contains(text(),"מינימום")]/following-sibling::label[1]')
            mitcham_data["Area m2"] = get_xpath_text('.//label[contains(text(),"שטח")]/following-sibling::label[1]')

            # Extract tables (Plan, Parcel, Block, Section)
            tables = mitcham.find_elements(By.TAG_NAME, 'table')
            if len(tables) >= 2:
                rows1 = tables[0].find_elements(By.TAG_NAME, 'tr')
                if len(rows1) >= 2:
                    cells1 = rows1[1].find_elements(By.TAG_NAME, 'td')
                    if len(cells1) >= 2:
                        mitcham_data["Plan"] = cells1[0].text.strip()
                        mitcham_data["Parcel"] = cells1[1].text.strip()
                        print(f"✅ Plan: {mitcham_data['Plan']}, Parcel: {mitcham_data['Parcel']}")

                rows2 = tables[1].find_elements(By.TAG_NAME, 'tr')
                if len(rows2) >= 2:
                    cells2 = rows2[1].find_elements(By.TAG_NAME, 'td')
                    if len(cells2) >= 2:
                        mitcham_data["Block"] = cells2[0].text.strip()
                        mitcham_data["Section"] = cells2[1].text.strip()
                        print(f"✅ Block: {mitcham_data['Block']}, Section: {mitcham_data['Section']}")
                        print(f"number of mitham: {counter}")
                        counter +=1 

            
            all_mitcham_data.append(mitcham_data)

        except Exception as e:
            print(f"❌ Error parsing mitcham: {str(e)}")
            continue

    return all_mitcham_data


In [29]:
def main_function():
    """
    Main function for running the entire scraping process.
    """
    driver = webdriver.Chrome()
    driver.get('https://apps.land.gov.il/MichrazimSite/#/homePage')

    driver.find_elements(By.XPATH, '//*[@class="fw-bolder button-enter w-50"]')[1].click()
    time.sleep(1)

    # Click on "Advanced Search"
    driver.find_elements(By.XPATH, '//*[@class="advanced-search"]')[0].click()
    time.sleep(1)

    # Apply filters
    micrhaz_type(driver)
    region_rmi(driver)
    micrhaz_purpose(driver)
    micrhaz_date(driver)

    # Scroll to load all results
    scroll_to_load_all_results(driver,type=0)
    time.sleep(2)

    results = driver.find_elements(By.XPATH, '//div[contains(@class, "list-rows list-group-item align-left ng-star-inserted")]')
    print(f"Number of tenders found: {len(results)}")

    all_tenders_data = []

    for i in range(len(results)):
        try:
            WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located(
                    (By.XPATH, '//div[contains(@class, "list-rows list-group-item align-left ng-star-inserted")]')
                )
            )

            # Refresh result elements in case the DOM changed
            results = driver.find_elements(By.XPATH, '//div[contains(@class, "list-rows list-group-item align-left ng-star-inserted")]')

            if i >= len(results):
                print(f"Warning: Index {i} out of bounds. Total results: {len(results)}")
                break

            try:
                # Try to find and click the main link
                link = results[i].find_element(By.XPATH, './/span[@class="michraz-link"]')
                driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", link)
                time.sleep(1)
                link.click()

            except Exception as link_error:
                print(f"Could not click on the main link: {str(link_error)}")

                # Try alternative method (by role)
                try:
                    button = results[i].find_element(By.XPATH, './/span[@role="button"]')
                    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", button)
                    time.sleep(1)
                    button.click()
                    print("Fallback button clicked successfully.")
                except Exception as button_error:
                    print(f"Fallback button click failed: {str(button_error)}")
                    continue

            time.sleep(3)

            current_url = driver.current_url
            print(f"Current URL: {current_url}")
            time.sleep(3)

            # Extract tender data
            tender_data_list = extract_tender_data(driver)
            all_tenders_data.extend(tender_data_list)


            # Return to main page
            driver.back()
            time.sleep(3)

        except Exception as e:
            print(f"Error processing tender {i + 1}: {str(e)}")
            try:
                driver.back()
                time.sleep(3)
            except:
                print("Failed to return to results page.")

    # Create DataFrame
    df = pd.DataFrame(all_tenders_data)

    # Save to CSV
    csv_filename = "FinalProject_Scraping/data/rmi_results_05_10_south.csv"
    try:
        df.to_csv(csv_filename, index=False, encoding='utf-8-sig')
        print(f"Data successfully saved to: {csv_filename}")
    except Exception as save_error:
        print(f"Failed to save CSV: {str(save_error)}")

    driver.quit()


In [30]:
main_function()

Scroll 1: found 20 tenders
Scroll 2: found 30 tenders
Scroll 3: found 40 tenders
Scroll 4: found 60 tenders
Scroll 5: found 67 tenders
Scroll 6: found 67 tenders
Scroll 7: found 67 tenders
Scroll 8: found 67 tenders
No more results found. Scrolling finished.
Number of tenders found: 67
Current URL: https://apps.land.gov.il/MichrazimSite/#/michraz/20090163
📄 Tender Number: 163/2009
✅ Found field: 'Region' = 'דרום'
✅ Found field: 'City' = 'באר אורה , באר אורה'
✅ Found field: 'Purpose' = 'בנייה נמוכה/צמודת קרקע'
✅ Found field: 'Publication date' = '16/07/2009'
✅ Found field: 'Status' = 'נדון בוועדת מכרזים'
✅ Found field: 'Opening date' = '29/07/2009'
✅ Found field: 'Submission deadline' = '06/09/2009 12:00'
✅ Found field: 'Committee date' = '09/09/2009'
Scroll 1: found 20 tenders
Scroll 2: found 30 tenders
Scroll 3: found 35 tenders
Scroll 4: found 35 tenders
Scroll 5: found 35 tenders
Scroll 6: found 35 tenders
No more results found. Scrolling finished.
✅ Loaded: .//label[contains(text()

KeyboardInterrupt: 

In [126]:
#mitcham_data["Units"] = mitcham.find_element(By.XPATH, ".//label[contains(text(),'יח\"ד')]/following-sibling::label[1]").text.strip()
